In [1]:
!pip install gradio transformers torch accelerate bitsandbytes pyngrok -q


In [2]:
!pip install timeout-decorator -q

  Preparing metadata (setup.py) ... done


In [3]:
from google.colab import files
uploaded = files.upload()


Saving smartsdlc_gradio_shared.py to smartsdlc_gradio_shared.py


In [ ]:
import gradio as gr
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import logging
import time
import timeout_decorator  # For adding a timeout to model loading

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Configuration parameters
config = {
    "output_dir": "/content/output",
    "max_file_size": 1048576,  # 1 MB
    "model_path": "ibm-granite/granite-3.0-2b-instruct",
    "use_quantization": True
}

# Function to read content from a Python file
def read_python_file(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception as e:
        logger.error(f"Error reading file '{file_path}': {str(e)}")
        return None

# Function to load the model with a timeout
@timeout_decorator.timeout(300)  # Timeout after 5 minutes
def load_granite_model():
    logger.info(f"Loading model: {config['model_path']}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(config["model_path"])
        kwargs = {
            "device_map": "auto",
            "torch_dtype": torch.float16 if torch.cuda.is_available() else torch.float32
        }
        if config["use_quantization"]:
            kwargs["quantization_config"] = BitsAndBytesConfig(load_in_4bit=True)
        model = AutoModelForCausalLM.from_pretrained(config["model_path"], **kwargs)
        model.eval()
        logger.info("Model loaded successfully.")
        return model, tokenizer
    except Exception as e:
        logger.critical(f"Model load failed: {str(e)}")
        return None, None

# Load the model with a try-catch for timeout
try:
    model, tokenizer = load_granite_model()
except timeout_decorator.TimeoutError:
    logger.critical("Model loading timed out after 5 minutes.")
    model, tokenizer = None, None

# Handle file uploads
def handle_file(file):
    if file is None:
        return None, "Error: No file uploaded."
    try:
        file_path = file.name if hasattr(file, 'name') else str(file)
        temp_file_path = f"/content/{os.path.basename(file_path)}_{int(time.time())}.py"
        with open(file_path, "rb") as f:
            file_content = f.read()
        with open(temp_file_path, "wb") as f:
            f.write(file_content)
        return temp_file_path, None
    except Exception as e:
        return None, f"Error processing file: {str(e)}"

# Core LLM generation function with reduced max tokens for speed
def granite_generate(prompt, model_instance, tokenizer_instance, max_tokens=512):  # Reduced for faster inference
    if model_instance is None or tokenizer_instance is None:
        return None
    try:
        inputs = tokenizer_instance(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")
        output = model_instance.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer_instance.eos_token_id
        )
        decoded_output = tokenizer_instance.decode(output[0], skip_special_tokens=True)
        return decoded_output[len(prompt):].strip() if decoded_output.startswith(prompt) else decoded_output.strip()
    except Exception as e:
        logger.error(f"Error during generation: {str(e)}")
        return None

# Generic code processing function
def process_code_with_llm(file, prompt_template):
    path, err = handle_file(file)
    if err:
        return err
    code_content = read_python_file(path)
    if code_content is None:
        return "Error: Could not read the file content."
    if model is None or tokenizer is None:
        return "Error: AI model is not loaded."
    prompt = prompt_template.format(code=code_content)
    generated_text = granite_generate(prompt, model, tokenizer)
    return generated_text if generated_text else "Generation failed."

# Gradio interface functions
def generate_docstrings_gradio(file):
    return process_code_with_llm(file, "Add Google-style docstrings to the following Python code:\n\n{code}")

def review_code_gradio(file):
    return process_code_with_llm(file, "Review the following Python code for PEP8 issues, logic errors, potential bugs, and suggestions for improvements:\n\n{code}")

def predict_bugs_gradio(file):
    return process_code_with_llm(file, "Identify and explain any bugs, security vulnerabilities, or potential issues in this Python code:\n\n{code}")

def generate_report_gradio(file):
    return process_code_with_llm(file, "Generate a comprehensive project report for the following Python code. The report should include: Purpose, Key Functions, Overall Structure, and areas for Future Improvements and Optimization:\n\n{code}")

# Gradio UI
with gr.Blocks() as demo:
    gr.Markdown("# 🚀 SmartSDLC: AI-Powered Software Development Assistant")
    gr.Markdown("Upload a Python file for AI assistance.")
    file_input = gr.File(label="📂 Upload a Python (.py) File", file_types=[".py"], height=80)
    with gr.Row():
        doc_btn = gr.Button("📝 Generate Docstrings")
        review_btn = gr.Button("🔍 Review Code")
        bug_btn = gr.Button("🐞 Predict Bugs")
        report_btn = gr.Button("📊 Generate Project Report")
    output_textbox = gr.Textbox(label="🧠 AI Output", lines=25, interactive=False, placeholder="Output will appear here...")
    doc_btn.click(fn=generate_docstrings_gradio, inputs=file_input, outputs=output_textbox)
    review_btn.click(fn=review_code_gradio, inputs=file_input, outputs=output_textbox)
    bug_btn.click(fn=predict_bugs_gradio, inputs=file_input, outputs=output_textbox)
    report_btn.click(fn=generate_report_gradio, inputs=file_input, outputs=output_textbox)
    demo.launch(share=True, debug=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f524f3664edfafff31.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:463: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [ ]:
gradio deploy